In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential,load_model,Model
from keras.layers import Dense,Conv2D,Input,DepthwiseConv2D,ReLU,Add,GlobalAveragePooling2D,BatchNormalization
from keras.callbacks import ModelCheckpoint
import cv2
import numpy as np
import tensorflow as tf

In [2]:
# Define the data generator with rescaling and validation split
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)

# Define the directories for the training and testing data
dir_train = 'D:\\Data sets\\Natural Human Face Images for Emotion Recognition\\train'
dir_test = 'D:\\Data sets\\Natural Human Face Images for Emotion Recognition\\test'

# Define the image size for the model
img_size=[224,224]


In [3]:
checkpoint = ModelCheckpoint(
    filepath='mobilenetv2_trained_model.h5',
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

In [4]:
train_data = datagen.flow_from_directory(
    dir_train,
    target_size=img_size,
    batch_size=16,
    class_mode='categorical',
    subset='training',
    color_mode='grayscale'
)
val_data = datagen.flow_from_directory(
    dir_train,
    target_size=img_size,
    batch_size=16,
    class_mode='categorical',
    subset='validation',
    color_mode='grayscale'
)
print(val_data.class_indices)

Found 4993 images belonging to 8 classes.
Found 550 images belonging to 8 classes.
{'anger': 0, 'contempt': 1, 'disgust': 2, 'fear': 3, 'happiness': 4, 'neutrality': 5, 'sadness': 6, 'surprise': 7}


In [5]:
def inverted_residual_block(inputs, filters, expansion_factor, strides):
    x = inputs
    # Pointwise convolution
    x = Conv2D(filters * expansion_factor, kernel_size=1, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Depthwise convolution
    x = DepthwiseConv2D(kernel_size=3, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Pointwise convolution
    x = Conv2D(filters, kernel_size=1, strides=1, padding='same')(x)
    x = BatchNormalization()(x)

    if strides == 1:
        x = Add()([inputs, x])

    return x

In [6]:
def pointwise_con(inputX,filters):
    x = Conv2D(filters, kernel_size=1, strides=1, padding='same')(inputX)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x

In [7]:
def MobileNetV2(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # Initial convolution layer
    x = Conv2D(32, kernel_size=3, strides=2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
      # Extra pointwise convolution layer to match the number of filters
    x = pointwise_con(x,16)

    # Inverted residual blocks
    # (inputs, filters, expansion_factor, strides)
    # (inputs, c, t, s)
    x = inverted_residual_block(x, 16, 1, 1)
    x = inverted_residual_block(x, 24, 6, 2)
    x = inverted_residual_block(x, 24, 6, 1)
    x = inverted_residual_block(x, 32, 6, 2)
    x = inverted_residual_block(x, 32, 6, 1)
    x = inverted_residual_block(x, 32, 6, 1)
    x = inverted_residual_block(x, 64, 6, 2)
    x = inverted_residual_block(x, 64, 6, 1)
    x = inverted_residual_block(x, 64, 6, 1)
    x = inverted_residual_block(x, 64, 6, 1)
    x = inverted_residual_block(x, 64, 6, 1)
    x = pointwise_con(x,96)
    x = inverted_residual_block(x, 96, 6, 1)
    x = inverted_residual_block(x, 96, 6, 1)
    x = inverted_residual_block(x, 160, 6, 2)
    x = inverted_residual_block(x, 160, 6, 1)
    x = inverted_residual_block(x, 160, 6, 1)
    x = pointwise_con(x,320)
    x = inverted_residual_block(x, 320, 6, 1)

    # Final convolution layer
    x = Conv2D(1280, kernel_size=1, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Global average pooling and output layer
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    return Model(inputs=inputs, outputs=outputs)

In [8]:
input_shape = (224, 224, 1)
num_classes = 8
model = MobileNetV2(input_shape, num_classes)

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 32  320         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 112, 112, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [10]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [11]:
model.fit(train_data, epochs=40, validation_data=val_data,callbacks=[checkpoint])

Epoch 1/40
313/313 [==============================] - ETA: 0s - loss: 1.9529 - accuracy: 0.2786
Epoch 1: val_accuracy improved from -inf to 0.13455, saving model to mobilenetv2_trained_model.h5
313/313 [==============================] - 68s 190ms/step - loss: 1.9529 - accuracy: 0.2786 - val_loss: 2.3739 - val_accuracy: 0.1345
Epoch 2/40
313/313 [==============================] - ETA: 0s - loss: 1.7362 - accuracy: 0.3519
Epoch 2: val_accuracy improved from 0.13455 to 0.13636, saving model to mobilenetv2_trained_model.h5
313/313 [==============================] - 57s 181ms/step - loss: 1.7362 - accuracy: 0.3519 - val_loss: 2.1261 - val_accuracy: 0.1364
Epoch 3/40
313/313 [==============================] - ETA: 0s - loss: 1.5812 - accuracy: 0.4178
Epoch 3: val_accuracy improved from 0.13636 to 0.35818, saving model to mobilenetv2_trained_model.h5
313/313 [==============================] - 56s 180ms/step - loss: 1.5812 - accuracy: 0.4178 - val_loss: 2.1510 - val_accuracy: 0.3582
Epoch 4/40

In [16]:
img=cv2.imread("D:\\Data sets\\Natural Human Face Images for Emotion Recognition\\test\\images - 2020-11-06T003457.012_face.png",cv2.IMREAD_GRAYSCALE)
# cv2.imshow('image',img)
# cv2.waitKey(0)
img_resized=cv2.resize(img,img_size)
img=np.expand_dims(img,axis=0)
img=np.expand_dims(img,axis=-1)
img=img/255

In [17]:
class_names=['anger', 'contempt', 'disgust', 'fear', 'happiness', 'neutrality', 'sadness', 'surprise']
model = load_model('mobilenetv2_trained_model.h5')
pred=model.predict(img)
output_class = class_names[np.argmax(pred)]
print("predicted  class is : ",output_class)

1/1 [==============================] - 1s 790ms/step
predicted  class is :  disgust


In [18]:
def create_tflite():
    file_name= 'emotion_detection_model.tflite'
    converter= tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    open(file_name,'wb').write(tflite_model)


In [19]:
create_tflite()

INFO:tensorflow:Assets written to: C:\Users\MDW\AppData\Local\Temp\tmp2cm1g_5n\assets


INFO:tensorflow:Assets written to: C:\Users\MDW\AppData\Local\Temp\tmp2cm1g_5n\assets
